In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from collections import OrderedDict
from functools import partial

from tqdm.contrib.concurrent import process_map
# from multiprocessing import Pool

import aoc
from rangemap import RangeMap

In [5]:
input = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [6]:
almanac = lambda input: OrderedDict({
    m.split('map:\n')[0].strip(): RangeMap.from_starts_and_lengths_str(m.split('map:\n')[1])
    for m in input.strip().split('\n\n')
    if not m.startswith('seeds:')
})
almanac(input)

OrderedDict([('seed-to-soil',
              (98, 99) -> (50, 51)
              (50, 97) -> (52, 99)),
             ('soil-to-fertilizer',
              (15, 51) -> (0, 36)
              (52, 53) -> (37, 38)
              (0, 14) -> (39, 53)),
             ('fertilizer-to-water',
              (53, 60) -> (49, 56)
              (11, 52) -> (0, 41)
              (0, 6) -> (42, 48)
              (7, 10) -> (57, 60)),
             ('water-to-light',
              (18, 24) -> (88, 94)
              (25, 94) -> (18, 87)),
             ('light-to-temperature',
              (77, 99) -> (45, 67)
              (45, 63) -> (81, 99)
              (64, 76) -> (68, 80)),
             ('temperature-to-humidity',
              (69, 69) -> (0, 0)
              (0, 68) -> (1, 69)),
             ('humidity-to-location',
              (56, 92) -> (60, 96)
              (93, 96) -> (56, 59))])

In [7]:
def seed_to(seed, dest_map_name, almanac, debug=False):
    index = seed
    if debug: print(f"seed {seed}", end=" ")
    for key, correspondance in almanac.items():
        index = correspondance[index]
        if debug: print(f"-> {key.split('-')[-1]} {index}", end=" ")
        if key == dest_map_name:
            return index
    return index

In [8]:
seed_to(48, "humidity-to-location", almanac(input), debug=True)

seed 48 -> soil 48 -> fertilizer 33 -> water 22 -> light 92 -> temperature 60 -> humidity 61 -> location 65 

65

In [9]:
def seed_to_location(seed, almanac):
    return seed_to(seed, "humidity-to-location", almanac)

In [10]:
seed_to_location(48, almanac(input))

65

In [11]:
seeds = lambda input: list(map(int, input.strip().split('\n')[0].split(':')[1].strip().split()))
seeds(input)

[79, 14, 55, 13]

In [12]:
def closest_location(seeds, almanac):
    return min([seed_to_location(seed, almanac) for seed in seeds])

In [13]:
closest_location(seeds(input), almanac(input))

35

In [14]:
input = aoc.input(5)
answer = closest_location(seeds(input), almanac(input))
answer

282277027

In [15]:
list(zip(seeds(input)[::2], seeds(input)[1::2]))

[(1187290020, 247767461),
 (40283135, 64738286),
 (2044483296, 66221787),
 (1777809491, 103070898),
 (108732160, 261552692),
 (3810626561, 257826205),
 (3045614911, 65672948),
 (744199732, 300163578),
 (3438684365, 82800966),
 (2808575117, 229295075)]

In [16]:
def closest_location(seed_range, almanac):
    result = process_map(
        partial(seed_to_location, almanac=almanac),
        range(seed_range[0], seed_range[0]+seed_range[1]),
        total=seed_range[1],
        chunksize=5)
    return min(result)

In [17]:
input = aoc.input(5)
answer = closest_location((1187290020, 247767461), almanac(input))
answer

C:\Users\yelkh\AppData\Local\Temp\ipykernel_20752\1083351717.py:2: TqdmWarning: Iterable length 247767461 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  result = process_map(
